In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import InceptionResNetV2,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexPE"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = InceptionResNetV2(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1536)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1536)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.1min


[CV] ........................... C=0.01, score=0.872070, total= 1.1min
[CV] ........................... C=0.01, score=0.891389, total= 1.1min
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.866142, total= 1.1min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.891972, total= 1.1min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.884273, total= 1.1min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.898406, total= 1.4min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.886680, total= 1.4min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  6.4min


[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.909990, total= 6.4min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.911133, total= 6.6min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.920588, total= 6.7min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.903543, total= 6.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.911706, total= 6.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.912402, total= 6.9min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.925636, total= 6.9min
[CV] C

...

[CV] ................... C=1000000000.0, score=0.898622, total= 8.0min
[CV] .................... C=100000000.0, score=0.908549, total= 8.5min
[CV] .................. C=10000000000.0, score=0.907227, total= 6.8min
[CV] .................. C=10000000000.0, score=0.910803, total= 6.1min
[CV] ................... C=1000000000.0, score=0.913776, total= 7.8min
[CV] ................... C=1000000000.0, score=0.903320, total= 8.7min
[CV] .................... C=100000000.0, score=0.920319, total= 8.8min
[CV] ................... C=1000000000.0, score=0.907843, total= 8.7min
[CV] ................... C=1000000000.0, score=0.919765, total= 8.7min
[CV] .................. C=10000000000.0, score=0.913690, total= 6.3min
[CV] ................... C=1000000000.0, score=0.906746, total= 8.2min
[CV] ................... C=1000000000.0, score=0.911531, total= 8.1min
[CV] ................... C=1000000000.0, score=0.916335, total= 8.1min
[CV] .................. C=10000000000.0, score=0.916830, total= 7.5min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 48.2min remaining:  1.9min


[CV] .................. C=10000000000.0, score=0.891732, total= 7.1min
[CV] .................. C=10000000000.0, score=0.899110, total= 6.9min
[CV] .................. C=10000000000.0, score=0.914370, total= 7.0min
[CV] .................. C=10000000000.0, score=0.913519, total= 6.5min
[CV] .................. C=10000000000.0, score=0.922311, total= 5.9min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 48.7min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 69.86496792, 404.94089608, 435.07041078, 573.61050332,
        640.43627887, 644.70460393, 659.57413471, 645.4391428 ,
        652.78257422, 650.0503644 , 597.30379705, 485.90284452,
        399.14787369]),
 'mean_score_time': array([0.04270899, 0.07728381, 0.0560379 , 0.06047397, 0.05123703,
        0.04225307, 0.05291483, 0.04637408, 0.05119178, 0.0370317 ,
        0.03838067, 0.01958287, 0.01749837]),
 'mean_test_score': array([0.88022889, 0.91406867, 0.91663378, 0.91160221, 0.9090371 ,
        0.90893844, 0.90962904, 0.90913575, 0.90913575, 0.90962904,
        0.91101026, 0.90953039, 0.9097277 ]),
 'mean_train_score': array([0.90679038, 0.96885672, 0.99572492, 0.99860799, 0.99826855,
        0.99810349, 0.99829024, 0.99853134, 0.99849853, 0.99854226,
        0.99857523, 0.99842178, 0.99836688]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.88023, std: 0.01269, params: {'C': 0.01},
 mean: 0.91407, std: 0.00897, params: {'C': 0.1},
 mean: 0.91663, std: 0.00906, params: {'C': 1.0},
 mean: 0.91160, std: 0.00682, params: {'C': 10.0},
 mean: 0.90904, std: 0.00779, params: {'C': 100.0},
 mean: 0.90894, std: 0.00706, params: {'C': 1000.0},
 mean: 0.90963, std: 0.00693, params: {'C': 10000.0},
 mean: 0.90914, std: 0.00679, params: {'C': 100000.0},
 mean: 0.90914, std: 0.00626, params: {'C': 1000000.0},
 mean: 0.90963, std: 0.00723, params: {'C': 10000000.0},
 mean: 0.91101, std: 0.00759, params: {'C': 100000000.0},
 mean: 0.90953, std: 0.00656, params: {'C': 1000000000.0},
 mean: 0.90973, std: 0.00841, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 1.0} - score: 0.9166
